In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing

# 1. Read and load DFs
This should be executed once and then use the serialized (pickeled) ones.

## 1.1 Using serialized DFs for time saving 

In [3]:
df2 = pd.read_pickle('datasets/raw/RAW2_reviews.pkl')

In [4]:
df3 = pd.read_pickle('datasets/raw/raw_recipes_w_search_terms.pkl')

### Analysing columns and datastructures

### Set unwanted columns and drop those

In [5]:
#df1_unwanted_columns = ["name", "tags", "n_steps", "steps", "description", "ingredients", "n_ingredients"]
df2_unwanted_columns = ["AuthorId", "Name", "Keywords", "Description", "CookTime", "PrepTime", "RecipeIngredientQuantities", "RecipeIngredientParts", "FatContent", "SaturatedFatContent", "CholesterolContent", "SodiumContent", "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent", "RecipeServings", "RecipeYield", "RecipeInstructions"]
df3_unwanted_columns = []

In [7]:
#df1.drop(df1_unwanted_columns, axis=1, inplace=True)
df2.drop(df2_unwanted_columns, axis=1, inplace=True)

### Check DFs

In [8]:
df2[df2["id"] == 137739]

,id,AuthorName,TotalTime,DatePublished,Images,RecipeCategory,AggregatedRating,ReviewCount,Calories
131151,137739,COOKGIRl,PT55M,2005-09-16T15:24:00Z,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,5.0,3.0,51.5


In [9]:
df3[df3["id"] == 137739]

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
63363,137739,! Arriba ! Baked Winter Squash Mexican-Style,Autumn is my favorite time of year to cook! Th...,"['winter squash', 'mexican seasoning', 'mixed ...","[""1 lb winter squash (such as hubbard, ac...",1 (129 g),3,['*NOTE: Two separate seasoning suggestions ar...,"['60-minutes-or-less', 'time-to-make', 'course...","{'mexican', 'side', 'vegetarian', 'baked'}"


### Conclusion about the 3 dataframes
Here can be seen, that the first dataset contains only nutrions that are not in the other ones, bt is is not compulsory.
The df1 can be left.

### Patching total time

In [10]:
df2['TotalTime'] = df2['TotalTime'].str[2:-1].str.split('H')
df2['TotalTime'] = df2['TotalTime'].apply(lambda x: (int(x[0]) * 60 + int(x[1])) if (len(x) > 1) else x[0])

In [11]:
df2['TotalTime']

0         1485
1          265
2           35
3         1460
4           50
          ... 
522512      95
522513     210
522514       4
522515      15
522516      40
Name: TotalTime, Length: 522517, dtype: object

### Patching submitted

In [12]:
df2["DatePublished"] = df2['DatePublished'].str[:10]

In [13]:
df2["DatePublished"]

0         1999-08-09
1         1999-08-29
2         1999-09-05
3         1999-09-03
4         1999-09-19
             ...    
522512    2020-12-22
522513    2020-12-22
522514    2020-12-22
522515    2020-12-22
522516    2020-12-22
Name: DatePublished, Length: 522517, dtype: object

---

# 2. Merge

In [14]:
df_merged = pd.merge(df3, df2, on="id", how="left")

___

# 3. Refining

### Drop duplicates due to megre

### Simplify column names

In [15]:
df_merged.rename(columns={
    #"contributor_id":"author_id",
    "AuthorName":"author_name",
    #"Description":"description",
    "TotalTime":"minutes",
    "DatePublished":"submitted",
    "Images":"image_url",
    "RecipeCategory":"category",
    #"Keywords":"keywords",
    "AggregatedRating":"rating",
    "ReviewCount":"review_count",
    "Calories":"calories"
}, inplace=True)

### Reindexing

In [16]:
reindex_columns = ["id","name","author_name","submitted","image_url","minutes","description","category","tags","search_terms","steps","ingredients","ingredients_raw_str","serving_size","servings","calories","rating","review_count"]
df_merged = df_merged.reindex(columns=reindex_columns)

### Replace NaN values to 0

In [17]:
df_merged["minutes"].fillna(0, inplace=True)
df_merged["review_count"].fillna(0, inplace=True)

### Change data types

In [18]:
df_merged["minutes"] = df_merged["minutes"].astype(int)

In [19]:
df_merged["review_count"] = df_merged["review_count"].astype(int)

In [20]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494963 entries, 0 to 494962
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   494963 non-null  int64  
 1   name                 494963 non-null  object 
 2   author_name          494949 non-null  object 
 3   submitted            494949 non-null  object 
 4   image_url            494949 non-null  object 
 5   minutes              494963 non-null  int64  
 6   description          485365 non-null  object 
 7   category             494294 non-null  object 
 8   tags                 494963 non-null  object 
 9   search_terms         494963 non-null  object 
 10  steps                494963 non-null  object 
 11  ingredients          494963 non-null  object 
 12  ingredients_raw_str  494963 non-null  object 
 13  serving_size         494963 non-null  object 
 14  servings             494963 non-null  int64  
 15  review_count     

## 3.1 Dealing with false data
There are several records that are having multiple NaN values. They might be unnecessary.

In [21]:
len(df_merged[df_merged['description'].isna()])

9598

### Drop unwanted lines

### Patching missing images with NaN values

In [22]:
df_merged.loc[df_merged["image_url"] == "character(0)", "image_url"] = np.nan
#df_merged[df_merged["image_url"].isna()]

### Separating images
Keeping only the first one if there are many.

In [23]:
def image_separator(img):
    if (img is not np.nan) and (img[:2] == "c("):
        resu = img[3:].split('\",')[0]
        return (f'"{resu}"')
    return img

In [24]:
test_image = 'c("https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/goj8UrKTNSYVONgsYBeA_0S9A3808.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/3PwRRgWPQNARxEy98tJ6_0S9A3805.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/zgJ1MMCwQKeNGWlXs8E9_0S9A3827.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/cSdUyOFASsCuBEQR6ZX4_IMG_0257.JPG", \n"https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/1HoI6OmTTX6XhhNWfQHA_1512733305639530.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/pic7ZjJDF.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/pico1sqDg.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/pic8RZ4tX.jpg", \n"https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/piceIbNzr.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/pic3ezacq.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/pico5O6nj.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/picx5uHkq.jpg")'
test_image2 = np.nan

In [25]:
print((image_separator(test_image)))

"https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/20/75/25/goj8UrKTNSYVONgsYBeA_0S9A3808.jpg"


In [26]:
df_merged["image_url"] = df_merged.apply(lambda x: image_separator(x["image_url"]), axis=1)

### Change border to solid

In [27]:
df_merged["search_terms"] = df_merged["search_terms"].apply(lambda x: '[' + x[1:-1] + ']')

## Check

In [28]:
df_merged

,id,name,author_name,submitted,image_url,minutes,description,category,tags,search_terms,steps,ingredients,ingredients_raw_str,serving_size,servings,review_count,calories,rating
0,96313,Grilled Garlic Cheese Grits,Barb G.,2004-07-24,"""https://img.sndimg.com/food/image/upload/w_55...",65,"We love grits, this is another good way to ser...",Grains,"['time-to-make', 'course', 'main-ingredient', ...","['diabetic', 'low-calorie', 'vegetarian', 'low...","['I a sauce pan, bring water to a boil; slowly...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,3,144.8,5.0
1,232037,Simple Shrimp and Andouille Jambalaya,lisar,2007-06-03,NaN,45,"Simple, easy and very tasty for when you are i...",One Dish Meal,"['60-minutes-or-less', 'time-to-make', 'course...","['dinner', 'shrimp']","['In a food processor, pulse the onion, red pe...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,3,756.5,5.0
2,41090,black-and-white bean salad,spatchcock,2002-09-24,NaN,5,NaN,Black Beans,"['15-minutes-or-less', 'time-to-make', 'course...","['vegetarian', 'salad', 'side', 'dinner', 'veg...","['In a large bowl, combine beans, tomato, onio...","['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,2,159.0,4.5
3,60656,Crock Pot Italian Zucchini,Karen in KS,2003-04-22,NaN,370,This is a good recipe for weight watchers. It ...,Vegetable,"['weeknight', 'time-to-make', 'course', 'main-...","['side', 'vegetarian', 'italian']",['Put all ingredients in the crock pot and coo...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,1,47.1,4.0
4,232047,Beef Stew With Dried Cherries,ktdid,2007-06-03,NaN,165,This is a fabulous stew that came from one of ...,Stew,"['time-to-make', 'course', 'main-ingredient', ...",['dinner'],"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,0,691.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494958,276465,Blackberry Orange Scones,MsTeechur,2008-01-04,NaN,27,"The orange zest makes for a flavorful, not ove...",Scones,"['30-minutes-or-less', 'time-to-make', 'course...","['breakfast', 'bread']",['Sift about 2 cups of flour onto a piece of w...,"['unbleached flour', 'baking soda', 'butter', ...","[""2 1/2 cups unbleached flour"",""2 teaspo...",1 (83 g),8,2,244.8,3.5
494959,257796,Slow Cooker Garlic Chicken With Rosemary,AZRoxy63,2007-10-08,NaN,440,Delicious and easy!,Chicken,"['main-ingredient', 'preparation', 'occasion',...","['dinner', 'chicken']","['Place rosemary springs, 1 lemon half, celery...","['roasting chickens', 'lemons', 'rosemary spri...","[""1 (5 lb) roasting chickens, rinsed and ...",1 (340 g),6,1,566.2,4.0
494960,78003,Pot Roast with Port (Stove Top),Rick B2,2003-12-05,NaN,115,This is a recipe from the Frugal Gourmet cooki...,Weeknight,"['weeknight', 'time-to-make', 'course', 'prepa...","['roast', 'dinner', 'low-sodium', 'low-carb']","['In a large pan, heat oil and brown roast on ...","['boneless beef chuck roast', 'olive oil', 'ta...","[""2 -3 lbs boneless beef chuck roast"",""2 ...",1 (283 g),4,5,828.2,5.0
494961,328810,Kapusta ( Cabbage and Kielbasa ),Nikki S,2008-10-04,NaN,2,Cabbage and sausage in tomato sauce,One Dish Meal,"['time-to-make', 'course', 'preparation', 'mai...",['dinner'],['Saute bacon in large pan until browned. Lea...,"['cabbage', 'condensed tomato soup', 'kielbasa...","[""8 cups cabbage or 2 heads cabbage,...",1 (564 g),6,0,688.0,NaN


--------

# 4. Save

In [29]:
df_merged.to_csv('datasets/refined/recipes_refined_merged.csv', sep='|', index=False)

In [30]:
df_merged.to_pickle('datasets/refined/recipes_refined_merged.pkl')